 # Школа DA_итоговый проект Казанцевой Дарьи

In [578]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor

In [431]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

/Users/nosocheeeek/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


----------Задача 1: Найти аэропорт с минимальной задержкой вылета

In [244]:
#Проверим столбец ORIGIN_AIRPORT на наличие пропусков
flights.ORIGIN_AIRPORT.isna().sum()

0

In [305]:
#Посмотрим, какие аэропорты, из которых отправляются рейсы, есть в датафрейме
flights.ORIGIN_AIRPORT.unique().tolist()
flights.DESTINATION_AIRPORT.unique().tolist()

In [ ]:
#В ORIGIN_AIRPORT и DESTINATION_AIRPORT встречаются обозначения аэропортов в разных кодировках.
#Нужно заменить пятизначные значения FAA в IATA, чтобы сделать данные однородными. 
#Для этого я нашла в Интернете два csv-файла, один содержит все аэропорты США в кодировке IATA, другой в FAA.

In [432]:
#Решаем проблему с несоответствующим кодом в колонке ORIGIN_AIRPORT
aircode1 = pd.read_csv('L_AIRPORT.csv')
aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

In [248]:
aircode1.head(5) #IATA кодировки

,Code,Description
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"


In [249]:
aircode2.head(5) #FAA кодировки

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"


In [433]:
#Преобразуем датафреймы для дальнейшего использования
aircode1 = aircode1.reset_index()
aircode2 = aircode2.reset_index()
aircodes = pd.merge(aircode1,aircode2,on='Description')
aircode_dict = dict(zip(aircodes['Code_y'].astype(str),aircodes['Code_x']))

flights['ORIGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].values.astype(str)
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

In [434]:
#Цикл для замены всех несоответсвующих IATA значений
for i in range(len(flights)):
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

In [252]:
#Для определения аэропорта с минимальной задержкой вылета нам не нужны несостоявшиеся вылеты
ncanceled_flights = flights.query('CANCELLED == False')

In [255]:
#Проверим столбец DEPARTURE_DELAY на наличие пропусков
ncanceled_flights.DEPARTURE_DELAY.isna().sum()

0

In [256]:
#Рассчитаем среднее время задержки вылета в каждом аэропорте
mean_delays = ncanceled_flights.groupby('ORIGIN_AIRPORT')['DEPARTURE_DELAY'].mean()

In [257]:
mean_delays.sort_values()

ORIGIN_AIRPORT
YAK    -6.070932
CNY    -6.058537
EKO    -3.933333
VEL    -3.740000
CDV    -3.454545
         ...    
OTH    18.969697
PPG    20.844828
HYA    23.182927
MVY    24.685185
ILG    29.568421
Name: DEPARTURE_DELAY, Length: 323, dtype: float64

In [258]:
min_delay_airport = mean_delays.idxmin()

In [259]:
min_delay_airport = airports.query('IATA_CODE == @min_delay_airport')

In [260]:
print(f'Аэропорт с минимальной задержкой вылета - {min_delay_airport.AIRPORT.iloc[0]}')

Аэропорт с минимальной задержкой вылета - Yakutat Airport


----------Задача №2: Найти самую пунктуальную авиакомпанию на прилет в Los Angeles International Airport

In [ ]:
#В DESTINATION_AIRPORT также встречается кодировка FAA. Исправим.

In [261]:
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

In [435]:
for i in range(len(flights)):
    if len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

In [267]:
#В ARRIVAL_DELAY есть пропуски. Заменим пропуски на среднее ARRIVAL_DELAY по авиакомпании.

In [436]:
flights.ARRIVAL_DELAY.isna().sum()

105071

In [544]:
delays = flights.groupby('AIRLINE')['ARRIVAL_DELAY'].mean().reset_index().copy()

In [545]:
for i in range(len(flights)):
    if math.isnan(flights['ARRIVAL_DELAY'][i]):
        airline = flights['AIRLINE'][i]
        flights['ARRIVAL_DELAY'][i] = delays.query('AIRLINE == @airline').iloc[0][1]

In [421]:
la_airport = airports.query('AIRPORT == "Los Angeles International Airport"')['IATA_CODE'].iloc[0]

In [422]:
punc = flights.query('DESTINATION_AIRPORT == @la_airport')

In [423]:
punc = pd.DataFrame(punc).reset_index()

In [424]:
#Если самолёт прилетел раньше запланированного, задержкой это считать не будем
punc.loc[punc['ARRIVAL_DELAY'] < 0, 'ARRIVAL_DELAY'] = 0

In [425]:
# Найдем среднюю задержку для каждой авиакомпании
avg_delays = punc.groupby('AIRLINE')['ARRIVAL_DELAY'].mean().reset_index()

# Сортируем по средней задержке и выбираем первую строку (самую пунктуальную авиакомпанию)
most_punctual = avg_delays.sort_values(by='ARRIVAL_DELAY').head(1)
most_punctual_airline = most_punctual['AIRLINE'].values[0]
airline = airlines.query('IATA_CODE == @most_punctual_airline').iloc[0][1]

print(f"Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport: {airline} с средней задержкой {most_punctual['ARRIVAL_DELAY'].values[0]:.2f} минут.")

Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport: Hawaiian Airlines Inc. с средней задержкой 6.19 минут.


----------Задача №3:Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [275]:
#Сделаем сводные таблицы по TAXI_OUT и TAXI_IN для каждого аэропорта, 
#затем объединим для высчитывания среднего времени на рулении для каждого аэропорта

t1 = flights.TAXI_IN.notna()
t2 = flights.TAXI_OUT.notna()

df = flights[t1&t2]

In [276]:
pivot_table1 = pd.pivot_table(
    df,
    index='ORIGIN_AIRPORT', 
    values=['TAXI_IN', 'FLIGHT_NUMBER'],  
    aggfunc={
        'TAXI_IN': 'sum',  
        'FLIGHT_NUMBER': 'count'  
    }
)

In [278]:
pivot_table2 = pd.pivot_table(
    df,
    index='DESTINATION_AIRPORT',  
    values=['TAXI_OUT', 'FLIGHT_NUMBER'], 
    aggfunc={
        'TAXI_OUT': 'sum',  
        'FLIGHT_NUMBER': 'count'  
    }
)

In [280]:
pivot_table1 = pivot_table1.reset_index().astype(str)
pivot_table2 = pivot_table2.reset_index().astype(str)
pivot_table1 = pivot_table1.rename(columns={'ORIGIN_AIRPORT': 'AIRPORT'})
pivot_table2 = pivot_table2.rename(columns={'DESTINATION_AIRPORT': 'AIRPORT'})

pivot = pd.merge(pivot_table1,pivot_table2,on = 'AIRPORT')

In [282]:
pivot['FLIGHT_NUMBER_x'] = pivot.FLIGHT_NUMBER_x.astype(int)
pivot['FLIGHT_NUMBER_y'] = pivot.FLIGHT_NUMBER_y.astype(int)
pivot['TAXI_IN'] = pivot.TAXI_IN.astype(float)
pivot['TAXI_OUT'] = pivot.TAXI_OUT.astype(float)

In [283]:
pivot['MEAN_TOTAL_TAXI'] = (pivot['TAXI_IN']+pivot['TAXI_OUT'])/(pivot['FLIGHT_NUMBER_x']+pivot['FLIGHT_NUMBER_y'])

In [285]:
pivot.MEAN_TOTAL_TAXI.isna().sum()

0

In [286]:
# Группировка по аэропорту назначения и расчёт среднего времени руления
airport_taxi_time = pivot.groupby('AIRPORT')['MEAN_TOTAL_TAXI'].mean()
airport_taxi_time

AIRPORT
ABE    13.654033
ABI    12.872292
ABQ    11.203859
ABR    12.960437
ABY    12.913043
         ...    
WRG     7.702113
WYS    13.084135
XNA    15.444814
YAK     7.814035
YUM    15.235858
Name: MEAN_TOTAL_TAXI, Length: 323, dtype: float64

In [419]:
# Поиск аэропорта с максимальным средним временем руления
max_taxi_airport = airport_taxi_time.idxmax()
max_taxi_airport = airports.query('IATA_CODE == @max_taxi_airport').iloc[0][1]
max_taxi_time = airport_taxi_time.max()

# Вывод результатов
print(f'Аэропорт с наибольшим средним временем руления: {max_taxi_airport}')
print(f'Среднее время руления: {max_taxi_time:.2f} минут')

Аэропорт с наибольшим средним временем руления: Sawyer International Airport
Среднее время руления: 19.88 минут


----------Задача №4.Построить модель, которая будет выбирать топ-3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета  

In [546]:
for_coder = flights[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT','ARRIVAL_DELAY']].copy()

In [547]:
# Кодирование категориальных признаков
le_origin = LabelEncoder()
le_dest = LabelEncoder()

for_coder['ORIGIN_AIRPORT_num'] = le_origin.fit_transform(for_coder['ORIGIN_AIRPORT'])
for_coder['DESTINATION_AIRPORT_num'] = le_dest.fit_transform(for_coder['DESTINATION_AIRPORT'])

In [490]:
# Выбор необходимых столбцов
X = pd.DataFrame(for_coder[['ORIGIN_AIRPORT_num', 'DESTINATION_AIRPORT_num']])
y = pd.DataFrame(for_coder['ARRIVAL_DELAY'])

In [491]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Сравним несколько моделей машинного обучения и выберем лучшую с помощью RMSE

In [552]:
# Создание и обучение линейной регрессионной модели
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse:.2f}')

RMSE: 39.08


In [573]:
# Создание и обучение DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse:.2f}')

RMSE: 38.86


In [ ]:
# DecisionTreeRegressor лучше, так как значение RMSE меньше

In [574]:
def get_top_3_destinations(origin):
    # Получаем индексы всех строк с указанным аэропортом вылета
    rows = X[X['ORIGIN_AIRPORT_num'] == origin]
    
    # Прогнозируем задержку для этих строк
    predictions = model.predict(rows)
    
    # Формируем словарь, где ключ - индекс строки, значение - предсказанная задержка
    delay_dict = dict(zip(rows.index, predictions))
    
    # Сортируем аэропорты назначения по возрастанию задержки
    sorted_delays = sorted(delay_dict.items(), key=lambda x: x[1])
    
    # Проверяем, что среди первых трёх аэропортов нет повторений
    unique_destinations = []
    seen_destinations = set()
    
    for idx, _ in sorted_delays:
        dest = X.at[idx, 'DESTINATION_AIRPORT_num']
        if dest not in seen_destinations:
            unique_destinations.append(dest)
            seen_destinations.add(dest)
        
        if len(unique_destinations) >= 3:
            break
            
    if len(unique_destinations) < 3:
        print('Невозможно построить топ-3 аэропорта прибытия для этого аэропорта отправления, так как их меньше 3.')
    
    return unique_destinations

In [577]:
origin = 'LAX'
decode_origin = airports[airports['IATA_CODE'] == origin].iloc[0][1]
origin_airport_num = for_coder[for_coder['ORIGIN_AIRPORT'] == origin].iloc[0][3]
top_3 = get_top_3_destinations(origin_airport_num)
decode_top_3 = []
for airport in top_3:
    decode_airport = for_coder[for_coder['DESTINATION_AIRPORT_num'] == airport].iloc[0][1]
    decode_airport = airports[airports['IATA_CODE'] == decode_airport].iloc[0][1]
    decode_top_3.append(decode_airport)
decode_top_3 = pd.DataFrame(decode_top_3)

print(" ")
print(f"Топ аэропортов назначения для вылета из {decode_origin}: {decode_top_3}")

 
Топ аэропортов назначения для вылета из Los Angeles International Airport:                                              0
0  Ted Stevens Anchorage International Airport
1           Philadelphia International Airport
2    Ronald Reagan Washington National Airport
